In [102]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit


# Defining Functions

In [103]:
def TrainTestSplit(data, test_size = 0.15, scale = False, cols_to_transform=None, include_test_scale=False):
    
    df = data.copy()
    # get the index after which test set starts
    test_index = int(len(df)*(1-test_size))
    
    # StandardScaler fit on the entire dataset
    if scale and include_test_scale:
        scaler = StandardScaler()
        df[cols_to_transform] = scaler.fit_transform(df[cols_to_transform])
        
    X_train = df.drop('demand', axis = 1).iloc[:test_index]
    y_train = df.demand.iloc[:test_index]
    X_test = df.drop('demand', axis = 1).iloc[test_index:]
    y_test = df.demand.iloc[test_index:]
    
    # StandardScaler fit only on the training set
    if scale and not include_test_scale:
        scaler = StandardScaler()
        X_train[cols_to_transform] = scaler.fit_transform(X_train[cols_to_transform])
        X_test[cols_to_transform] = scaler.transform(X_test[cols_to_transform])
    
    return X_train, X_test, y_train, y_test

# Importing Data

In [104]:
# Reading the gefcom dataset from rda file
df = pd.read_csv('./Data/cleandata/CleanedCT.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df

,ts,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working
0,2004-01-01 00:00:00,CT,3126.000,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working
1,2004-01-01 01:00:00,CT,2945.000,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working
2,2004-01-01 02:00:00,CT,2804.000,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working
3,2004-01-01 03:00:00,CT,2729.000,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working
4,2004-01-01 04:00:00,CT,2722.000,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113971,2016-12-31 19:00:00,CT,3744.918,40.0,29.0,2016-12-31,2016,12,19,Sat,366,True,False,121315.0,non-working
113972,2016-12-31 20:00:00,CT,3558.586,41.0,30.0,2016-12-31,2016,12,20,Sat,366,True,False,121316.0,non-working
113973,2016-12-31 21:00:00,CT,3378.466,38.0,32.0,2016-12-31,2016,12,21,Sat,366,True,False,121317.0,non-working
113974,2016-12-31 22:00:00,CT,3195.386,37.0,32.0,2016-12-31,2016,12,22,Sat,366,True,False,121318.0,non-working


In [105]:
df['day_of_week'] = df['day_of_week'].astype('category')
df['non_working'] = df['non_working'].astype('category')
df['month'] = df['month'].astype('category')
df['ts'] = pd.to_datetime(df['ts'])
CT =df.set_index('ts')
CT.head()

,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working
ts,,,,,,,,,,,,,,
2004-01-01 00:00:00,CT,3126.0,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working
2004-01-01 01:00:00,CT,2945.0,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working
2004-01-01 02:00:00,CT,2804.0,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working
2004-01-01 03:00:00,CT,2729.0,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working
2004-01-01 04:00:00,CT,2722.0,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working


In [106]:
def season_calc(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

In [107]:
CT.reset_index(inplace=True)
CT.head()

,ts,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working
0,2004-01-01 00:00:00,CT,3126.0,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working
1,2004-01-01 01:00:00,CT,2945.0,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working
2,2004-01-01 02:00:00,CT,2804.0,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working
3,2004-01-01 03:00:00,CT,2729.0,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working
4,2004-01-01 04:00:00,CT,2722.0,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working


In [108]:
#CT.reset_index(inplace=True)
#CT = CT.drop(['level_0', 'index'], axis=1)
CT['season'] = CT.ts.dt.month.apply(season_calc)
CT = CT.set_index(['ts'])
CT.head()

,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working,season
ts,,,,,,,,,,,,,,,
2004-01-01 00:00:00,CT,3126.0,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working,Winter
2004-01-01 01:00:00,CT,2945.0,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working,Winter
2004-01-01 02:00:00,CT,2804.0,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working,Winter
2004-01-01 03:00:00,CT,2729.0,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working,Winter
2004-01-01 04:00:00,CT,2722.0,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working,Winter


In [109]:
# Dividing the hours into 4 groups-> night, morning, afternoon, evening

hour_dict = {'morning': list(np.arange(7,13)),'afternoon': list(np.arange(13,16)), 'evening': list(np.arange(16,22)),
            'night': [22, 23, 0, 1, 2, 3, 4, 5, 6]}
hour_dict

{'morning': [7, 8, 9, 10, 11, 12],
 'afternoon': [13, 14, 15],
 'evening': [16, 17, 18, 19, 20, 21],
 'night': [22, 23, 0, 1, 2, 3, 4, 5, 6]}

In [110]:
def time_of_day(x):
    if x in hour_dict['morning']:
        return 'morning'
    elif x in hour_dict['afternoon']:
        return 'afternoon'
    elif x in hour_dict['evening']:
        return 'evening'
    else:
        return 'night'


In [111]:
CT['time_of_day'] = CT['hour'].apply(time_of_day)
CT.head()

,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working,season,time_of_day
ts,,,,,,,,,,,,,,,,
2004-01-01 00:00:00,CT,3126.0,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working,Winter,night
2004-01-01 01:00:00,CT,2945.0,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working,Winter,night
2004-01-01 02:00:00,CT,2804.0,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working,Winter,night
2004-01-01 03:00:00,CT,2729.0,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working,Winter,night
2004-01-01 04:00:00,CT,2722.0,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working,Winter,night


In [112]:
# creating categorical columns for linear regression 
cat_cols1 = ['month', 'day_of_year', 'hour', 'day_of_week', 'season', 'holiday', 'non_working', 'time_of_day']
#not including year above to capture the decreasing energy trend over increasing value of years
for col in cat_cols1:
    CT[col] = CT[col].astype('category')

In [113]:
CT['year'] = CT['year'].astype('int64')
CT.head()

,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working,season,time_of_day
ts,,,,,,,,,,,,,,,,
2004-01-01 00:00:00,CT,3126.0,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working,Winter,night
2004-01-01 01:00:00,CT,2945.0,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working,Winter,night
2004-01-01 02:00:00,CT,2804.0,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working,Winter,night
2004-01-01 03:00:00,CT,2729.0,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working,Winter,night
2004-01-01 04:00:00,CT,2722.0,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working,Winter,night


In [114]:
cols_use = ['demand', 'year', 'time_of_day', 'non_working', 'drybulb', 'dewpnt', 'season']
CT1_lin = pd.get_dummies(CT[cols_use], drop_first = True)
print(CT1_lin.shape)
CT1_lin.head()

(113976, 11)


,demand,year,drybulb,dewpnt,time_of_day_evening,time_of_day_morning,time_of_day_night,non_working_working,season_Spring,season_Summer,season_Winter
ts,,,,,,,,,,,
2004-01-01 00:00:00,3126.0,2004,33.0,26.0,0,0,1,0,0,0,1
2004-01-01 01:00:00,2945.0,2004,34.0,26.0,0,0,1,0,0,0,1
2004-01-01 02:00:00,2804.0,2004,40.0,26.0,0,0,1,0,0,0,1
2004-01-01 03:00:00,2729.0,2004,38.0,23.0,0,0,1,0,0,0,1
2004-01-01 04:00:00,2722.0,2004,37.0,21.0,0,0,1,0,0,0,1


In [115]:
from sklearn.ensemble import RandomForestRegressor

In [116]:
cols_to_transform = ['drybulb', 'dewpnt', 'year']
X_train, X_test, y_train, y_test = TrainTestSplit(CT1_lin, test_size = 0.15, scale = True, cols_to_transform=cols_to_transform, 
                                              include_test_scale=False)

In [117]:
X_train.head()

,year,drybulb,dewpnt,time_of_day_evening,time_of_day_morning,time_of_day_night,non_working_working,season_Spring,season_Summer,season_Winter
ts,,,,,,,,,,
2004-01-01 00:00:00,-1.580168,-0.949256,-0.660553,0,0,1,0,0,0,1
2004-01-01 01:00:00,-1.580168,-0.897289,-0.660553,0,0,1,0,0,0,1
2004-01-01 02:00:00,-1.580168,-0.585487,-0.660553,0,0,1,0,0,0,1
2004-01-01 03:00:00,-1.580168,-0.689421,-0.811779,0,0,1,0,0,0,1
2004-01-01 04:00:00,-1.580168,-0.741388,-0.912596,0,0,1,0,0,0,1


In [118]:
# Tuning Random forest
# n_estimators = number of trees in the forest
# max_features = max number of features considered for splitting a node

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(10, 200, 10, endpoint=True)]
max_features = ['auto', 'sqrt']
max_depth = list(range(1,6))
# Create the random grid
random_grid = {'n_estimators': n_estimators, 'max_features': max_features, 'max_depth':max_depth}
print(random_grid)

{'n_estimators': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [1, 2, 3, 4, 5]}


In [119]:
#import randomsearchcv
from sklearn.model_selection import RandomizedSearchCV

# First create the base model to tune
rf = RandomForestRegressor()

# Creating a time series split as discussed in the Introduction
tscv = TimeSeriesSplit(n_splits=5)
# Random search of parameters
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               cv = tscv, verbose=2, random_state = 42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

rf_random.best_params_
#rf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 73, 'max_features': 'auto', 'max_depth': 5}

# 1. Creating our population

In [120]:
CT.head()

,zone,demand,drybulb,dewpnt,date,year,month,hour,day_of_week,day_of_year,weekend,holiday,trend,non_working,season,time_of_day
ts,,,,,,,,,,,,,,,,
2004-01-01 00:00:00,CT,3126.0,33.0,26.0,2004-01-01,2004,1,0,Thu,1,False,True,7344.0,non-working,Winter,night
2004-01-01 01:00:00,CT,2945.0,34.0,26.0,2004-01-01,2004,1,1,Thu,1,False,True,7345.0,non-working,Winter,night
2004-01-01 02:00:00,CT,2804.0,40.0,26.0,2004-01-01,2004,1,2,Thu,1,False,True,7346.0,non-working,Winter,night
2004-01-01 03:00:00,CT,2729.0,38.0,23.0,2004-01-01,2004,1,3,Thu,1,False,True,7347.0,non-working,Winter,night
2004-01-01 04:00:00,CT,2722.0,37.0,21.0,2004-01-01,2004,1,4,Thu,1,False,True,7348.0,non-working,Winter,night


In [121]:
pop = CT.drop(['zone', 'date', 'day_of_week', 'day_of_year', 'weekend', 'holiday', 'trend', 'time_of_day', 'season', 'demand'], axis=1)
#CTtest = CTtest.drop(['index'], axis=1)
pop.head()

,drybulb,dewpnt,year,month,hour,non_working
ts,,,,,,
2004-01-01 00:00:00,33.0,26.0,2004,1,0,non-working
2004-01-01 01:00:00,34.0,26.0,2004,1,1,non-working
2004-01-01 02:00:00,40.0,26.0,2004,1,2,non-working
2004-01-01 03:00:00,38.0,23.0,2004,1,3,non-working
2004-01-01 04:00:00,37.0,21.0,2004,1,4,non-working


In [122]:
scaler = StandardScaler()
cols_to_transform = ['drybulb', 'dewpnt', 'year']
scalar = scaler.fit(pop[cols_to_transform])

## 1.1 Turning our pop into X_train format

In [123]:
pop_forDemandPred = pop.copy()
pop_forDemandPred['season'] = pop_forDemandPred['month'].apply(season_calc)
pop_forDemandPred['time_of_day'] = pop_forDemandPred['hour'].apply(time_of_day)
pop_forDemandPred.head()

,drybulb,dewpnt,year,month,hour,non_working,season,time_of_day
ts,,,,,,,,
2004-01-01 00:00:00,33.0,26.0,2004,1,0,non-working,Winter,night
2004-01-01 01:00:00,34.0,26.0,2004,1,1,non-working,Winter,night
2004-01-01 02:00:00,40.0,26.0,2004,1,2,non-working,Winter,night
2004-01-01 03:00:00,38.0,23.0,2004,1,3,non-working,Winter,night
2004-01-01 04:00:00,37.0,21.0,2004,1,4,non-working,Winter,night


In [124]:
cols_use = ['year', 'time_of_day', 'non_working', 'drybulb', 'dewpnt', 'season']
pop_forDemandPred = pd.get_dummies(pop_forDemandPred[cols_use], drop_first = True)
print(pop_forDemandPred.shape)
pop_forDemandPred.head()

(113976, 10)


,year,drybulb,dewpnt,time_of_day_evening,time_of_day_morning,time_of_day_night,non_working_working,season_Spring,season_Summer,season_Winter
ts,,,,,,,,,,
2004-01-01 00:00:00,2004,33.0,26.0,0,0,1,0,0,0,1
2004-01-01 01:00:00,2004,34.0,26.0,0,0,1,0,0,0,1
2004-01-01 02:00:00,2004,40.0,26.0,0,0,1,0,0,0,1
2004-01-01 03:00:00,2004,38.0,23.0,0,0,1,0,0,0,1
2004-01-01 04:00:00,2004,37.0,21.0,0,0,1,0,0,0,1


In [125]:
pop_forDemandPred[cols_to_transform] = scaler.transform(pop_forDemandPred[cols_to_transform])
pop_forDemandPred.head()

,year,drybulb,dewpnt,time_of_day_evening,time_of_day_morning,time_of_day_night,non_working_working,season_Spring,season_Summer,season_Winter
ts,,,,,,,,,,
2004-01-01 00:00:00,-1.603278,-0.957690,-0.663635,0,0,1,0,0,0,1
2004-01-01 01:00:00,-1.603278,-0.905916,-0.663635,0,0,1,0,0,0,1
2004-01-01 02:00:00,-1.603278,-0.595267,-0.663635,0,0,1,0,0,0,1
2004-01-01 03:00:00,-1.603278,-0.698816,-0.814972,0,0,1,0,0,0,1
2004-01-01 04:00:00,-1.603278,-0.750591,-0.915863,0,0,1,0,0,0,1


In [126]:
rf_random.predict(pop_forDemandPred)

array([2959.53981131, 2959.53981131, 2816.39900697, ..., 4156.03165591,
       2956.52386042, 2956.52386042])

In [127]:
def predict_demand(df):
    output = df.copy()
    pop_forDemandPred = df.copy()

    pop_forDemandPred['season'] = pop_forDemandPred['month'].apply(season_calc)
    pop_forDemandPred['time_of_day'] = pop_forDemandPred['hour'].apply(time_of_day)

    cols_use = ['year', 'time_of_day', 'non_working', 'drybulb', 'dewpnt', 'season']
    cols_to_transform = ['drybulb', 'dewpnt', 'year']
    pop_forDemandPred = pd.get_dummies(pop_forDemandPred[cols_use], drop_first = True)
    pop_forDemandPred[cols_to_transform] = scaler.transform(pop_forDemandPred[cols_to_transform])

    predicted_values = rf_random.predict(pop_forDemandPred)
    output['demand'] = predicted_values

    return output

In [128]:
predict_demand(pop)

,drybulb,dewpnt,year,month,hour,non_working,demand
ts,,,,,,,
2004-01-01 00:00:00,33.0,26.0,2004,1,0,non-working,2959.539811
2004-01-01 01:00:00,34.0,26.0,2004,1,1,non-working,2959.539811
2004-01-01 02:00:00,40.0,26.0,2004,1,2,non-working,2816.399007
2004-01-01 03:00:00,38.0,23.0,2004,1,3,non-working,2951.648058
2004-01-01 04:00:00,37.0,21.0,2004,1,4,non-working,2959.539811
...,...,...,...,...,...,...,...
2016-12-31 19:00:00,40.0,29.0,2016,12,19,non-working,4156.031656
2016-12-31 20:00:00,41.0,30.0,2016,12,20,non-working,4156.031656
2016-12-31 21:00:00,38.0,32.0,2016,12,21,non-working,4156.031656


# Selection

In [129]:
pop_small = pop.sample(1000)
print(pop_small.shape)
pop_small.reset_index(inplace=True, drop=True)
pop_small.head()

(1000, 6)


,drybulb,dewpnt,year,month,hour,non_working
0,52.0,48.0,2016,10,20,working
1,88.0,67.0,2012,8,13,working
2,30.0,29.0,2009,10,5,working
3,46.0,14.0,2012,4,9,non-working
4,31.0,7.0,2012,11,22,working


In [130]:
popwithDemand = predict_demand(pop_small)
popwithDemand

,drybulb,dewpnt,year,month,hour,non_working,demand
0,52.0,48.0,2016,10,20,working,3890.720506
1,88.0,67.0,2012,8,13,working,6070.109475
2,30.0,29.0,2009,10,5,working,3203.416322
3,46.0,14.0,2012,4,9,non-working,3393.207330
4,31.0,7.0,2012,11,22,working,2994.629028
...,...,...,...,...,...,...,...
995,40.0,38.0,2007,11,4,non-working,2807.468547
996,83.0,67.0,2005,6,10,non-working,4745.323204
997,52.0,38.0,2014,10,16,working,3890.720506
998,77.0,72.0,2009,8,19,non-working,3899.188926


In [131]:
def fitness(df):
    mu = df['demand'].mean()
    sd = df['demand'].std()

    Fitness = []
    for i in range(len(df)):
        z = ((df['demand'][i]) - mu) / sd
        Fitness.append(z)

    invFitness = []
    for i in range(len(Fitness)):
        x = Fitness[i] * (-1)
        invFitness.append(x)

    NormalFitness = []
    min = np.min(invFitness)
    max = np.max(invFitness)


    for i in range(len(invFitness)):
        n = (invFitness[i] - min) / (max - min)
        NormalFitness.append(n)

    df_Fitness = df.copy()
    df_Fitness['Fitness'] = NormalFitness

    return df_Fitness


In [132]:
eval = fitness(popwithDemand)
eval

,drybulb,dewpnt,year,month,hour,non_working,demand,Fitness
0,52.0,48.0,2016,10,20,working,3890.720506,0.644422
1,88.0,67.0,2012,8,13,working,6070.109475,0.000000
2,30.0,29.0,2009,10,5,working,3203.416322,0.847651
3,46.0,14.0,2012,4,9,non-working,3393.207330,0.791532
4,31.0,7.0,2012,11,22,working,2994.629028,0.909387
...,...,...,...,...,...,...,...,...
995,40.0,38.0,2007,11,4,non-working,2807.468547,0.964729
996,83.0,67.0,2005,6,10,non-working,4745.323204,0.391725
997,52.0,38.0,2014,10,16,working,3890.720506,0.644422
998,77.0,72.0,2009,8,19,non-working,3899.188926,0.641918


In [133]:
def roulette_selection(df):
    
    F = df['Fitness'].sum()
    df['SelectionProb'] = df['Fitness'] / F
    df['CumulativeProb'] = df['SelectionProb'].cumsum()

    selectors = np.random.random_sample((len(df),))

    dict_copy = df.to_dict('records')
    selectedChromsIndexes = []

    i = 0
    for selector in selectors:
        for r in dict_copy:
            if (r['CumulativeProb']) > selector:
                selectedChromsIndexes.append(r)
                break

    selected = pd.DataFrame(selectedChromsIndexes)

    return selected



In [134]:
selected = roulette_selection(eval)
selected

,drybulb,dewpnt,year,month,hour,non_working,demand,Fitness,SelectionProb,CumulativeProb
0,53.0,48.0,2008,6,6,working,2691.198920,0.999108,0.001421,0.744454
1,62.0,54.0,2016,9,8,working,3890.720506,0.644422,0.000916,0.438949
2,58.0,43.0,2015,11,21,working,3890.720506,0.644422,0.000916,0.415489
3,46.0,45.0,2009,4,9,working,3921.432409,0.635341,0.000903,0.286031
4,71.0,67.0,2014,7,22,working,3476.030257,0.767042,0.001091,0.690352
...,...,...,...,...,...,...,...,...,...,...
995,73.0,68.0,2008,9,0,non-working,3602.211888,0.729731,0.001038,0.815932
996,82.0,67.0,2013,8,16,working,5322.876896,0.220949,0.000314,0.309446
997,52.0,48.0,2014,9,4,working,2688.182969,1.000000,0.001422,0.964305
998,46.0,43.0,2012,10,8,working,3890.720506,0.644422,0.000916,0.915849


# Cross

In [135]:
select = selected.copy()
select['ProbCros'] = np.random.random(len(select))
select['Cross'] = select.apply(lambda x: 1 if (x['ProbCros'] < 0.25) else 0, axis=1)

In [136]:
select.head()

,drybulb,dewpnt,year,month,hour,non_working,demand,Fitness,SelectionProb,CumulativeProb,ProbCros,Cross
0,53.0,48.0,2008,6,6,working,2691.198920,0.999108,0.001421,0.744454,0.876638,0
1,62.0,54.0,2016,9,8,working,3890.720506,0.644422,0.000916,0.438949,0.055016,1
2,58.0,43.0,2015,11,21,working,3890.720506,0.644422,0.000916,0.415489,0.903722,0
3,46.0,45.0,2009,4,9,working,3921.432409,0.635341,0.000903,0.286031,0.851022,0
4,71.0,67.0,2014,7,22,working,3476.030257,0.767042,0.001091,0.690352,0.166635,1


In [137]:
import random

In [138]:
cross_point = random.randint(1,5)
cross_point

2

In [139]:
def sp_crossover(chrom1, chrom2):
    cross_point = random.randint(1,5)
    c1_l = chrom1[0:cross_point]
    c1_r = chrom1[cross_point:len(chrom1)]

    c2_l = chrom2[0:cross_point]
    c2_r = chrom2[cross_point:len(chrom2)]

    child1 = np.concatenate((c1_l, c2_r))
    child2 = np.concatenate((c2_l, c1_r))

    return child1, child2

In [140]:
def cross(df, crossProb = 0.25):
    select = df.copy()
    select['ProbCros'] = np.random.random(len(select))
    select['Cross'] = select.apply(lambda x: 1 if (x['ProbCros'] < crossProb) else 0, axis=1)
    
    cross = select.loc[select['Cross'] == 1]
    cross = cross.iloc[:, 0:6]
    cross['group'] = 1
    cross['group'].iloc[(int(len(cross) / 2)):] = 2

    if len(cross[cross['group'] == 1]) != len(cross[cross['group'] == 2]):
        cross = cross.iloc[:-1 , :]
    cross.drop(['group'], inplace=True, axis=1)

    group1 = cross.iloc[:(int(len(cross) / 2))].to_numpy()
    group2 = cross.iloc[(int(len(cross) / 2)):].to_numpy()

    crossedGen = []
    for i, j in zip(group1, group2):
        children = sp_crossover(i, j)
        crossedGen.append(children[0])
        crossedGen.append(children[1])

    cross['children'] = crossedGen
    cross[:] = cross.pop('children').to_list()

    for i, j in cross.iterrows():
        select.iloc[i, 0:6] = j

    return (select.iloc[:, :6])

In [141]:
crossed = cross(selected)
print('Are the selected and crossed the same?:', selected.equals(crossed))
crossed

/Users/alnoaimi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Are the selected and crossed the same?: False


,drybulb,dewpnt,year,month,hour,non_working
0,53.0,48.0,2008,6,6,working
1,62.0,54.0,2016,9,8,working
2,58.0,43.0,2015,9,6,non-working
3,46.0,45.0,2009,4,9,working
4,71.0,67.0,2014,7,22,working
...,...,...,...,...,...,...
995,73.0,68.0,2008,9,0,non-working
996,82.0,67.0,2013,8,16,working
997,52.0,48.0,2014,9,4,working
998,46.0,43.0,2012,10,8,working


# Mutation

In [142]:
# non-working Mutation
nonWorkingMutated = []
for i in crossed['non_working'].values:
    print(i)
    if (np.random.random()) < 0.01:
        if i == 'working':
            x = 'non-working'
            nonWorkingMutated.append(x)
        else:
            x = 'working'
            nonWorkingMutated.append(x)
    else:
        k = i
        nonWorkingMutated.append(k)
    break

nonWorkingMutated


working


['working']

In [145]:
def mutate(df, mutation_rate=0.01):
    newTEST = df.copy()

    drybulbMutated = []
    for i in newTEST['drybulb'].values:
        if (np.random.random()) < mutation_rate:
            x = np.random.randint((-11), 103)
            drybulbMutated.append(x)
        else:
            k = i
            drybulbMutated.append(k)

    newTEST['drybulb'] = drybulbMutated

    #dewpnt Mutation
    dewpntMutated = []
    for i in newTEST['dewpnt'].values:
        if (np.random.random()) < mutation_rate:
            x = np.random.randint((-27), 82)
            dewpntMutated.append(x)
        else:
            k = i
            dewpntMutated.append(k)

    newTEST['dewpnt'] = dewpntMutated

    # Year Mutation
    yearMutated = []
    for i in newTEST['year'].values:
        if (np.random.random()) < mutation_rate:
            x = np.random.randint(2004, 2017)
            yearMutated.append(x)
        else:
            k = i
            yearMutated.append(k)

    newTEST['year'] = yearMutated

    # Month Mutation
    monthMutated = []
    for i in newTEST['month'].values:
        if (np.random.random()) < mutation_rate:
            x = np.random.randint(1, 13)
            monthMutated.append(x)
        else:
            k = i
            monthMutated.append(k)

    newTEST['month'] = monthMutated

    # Hour Mutation
    hourMutated = []
    for i in newTEST['hour'].values:
        if (np.random.random()) < mutation_rate:
            x = np.random.randint(0, 24)
            hourMutated.append(x)
        else:
            k = i
            hourMutated.append(k)

    newTEST['hour'] = hourMutated

    # non-working Mutation
    nonWorkingMutated = []
    for i in crossed['non_working'].values:
        if (np.random.random()) < mutation_rate:
            if i == 'working':
                x = 'non-working'
                nonWorkingMutated.append(x)
            else:
                x = 'working'
                nonWorkingMutated.append(x)
        else:
            k = i
            nonWorkingMutated.append(k)

    newTEST['non_working'] = nonWorkingMutated

    return newTEST


In [146]:
mutated = mutate(crossed)
print('Are the crossed and mutated the same?:', crossed.equals(mutated))
mutated

Are the crossed and mutated the same?: False


,drybulb,dewpnt,year,month,hour,non_working
0,53.0,48.0,2008,6,6,working
1,62.0,54.0,2016,9,8,working
2,58.0,43.0,2015,9,6,non-working
3,46.0,45.0,2009,4,9,working
4,71.0,67.0,2014,7,22,working
...,...,...,...,...,...,...
995,73.0,68.0,2008,9,0,non-working
996,82.0,67.0,2013,8,16,working
997,52.0,48.0,2014,9,4,working
998,46.0,43.0,2012,10,8,working


# Putting it all together

In [150]:
def GA(df, NumOfGenerations= 50):
    pop = df.copy()
    for i in range(NumOfGenerations):
        popwithDemand = predict_demand(pop)
        eval = fitness(popwithDemand)
        selected = roulette_selection(eval)
        crossed = cross(selected)
        mutated = mutate(crossed, mutation_rate=0.1)
        pop = mutated.copy()

    return pop

In [156]:
FinalGen = GA(pop_small, NumOfGenerations=1000)

/Users/alnoaimi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/alnoaimi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/alnoaimi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

# Assesing the final Generation

In [157]:
pop_small

,drybulb,dewpnt,year,month,hour,non_working
0,52.0,48.0,2016,10,20,working
1,88.0,67.0,2012,8,13,working
2,30.0,29.0,2009,10,5,working
3,46.0,14.0,2012,4,9,non-working
4,31.0,7.0,2012,11,22,working
...,...,...,...,...,...,...
995,40.0,38.0,2007,11,4,non-working
996,83.0,67.0,2005,6,10,non-working
997,52.0,38.0,2014,10,16,working
998,77.0,72.0,2009,8,19,non-working


In [158]:
FinalGen

,drybulb,dewpnt,year,month,hour,non_working
0,7.0,58.0,2011,12,6,working
1,36.0,-8.0,2013,3,14,working
2,56.0,15.0,2013,5,5,non-working
3,56.0,60.0,2007,8,16,working
4,56.0,29.0,2010,3,2,working
...,...,...,...,...,...,...
995,51.0,25.0,2004,11,2,non-working
996,53.0,81.0,2010,12,2,working
997,45.0,12.0,2009,9,23,working
998,24.0,68.0,2005,10,18,working


In [159]:
finalDemand = predict_demand(FinalGen)
finalEval = fitness(finalDemand)
finalEval

,drybulb,dewpnt,year,month,hour,non_working,demand,Fitness
0,7.0,58.0,2011,12,6,working,3600.292479,0.635552
1,36.0,-8.0,2013,3,14,working,3890.720506,0.519506
2,56.0,15.0,2013,5,5,non-working,2688.182969,1.000000
3,56.0,60.0,2007,8,16,working,3921.432409,0.507235
4,56.0,29.0,2010,3,2,working,2688.182969,1.000000
...,...,...,...,...,...,...,...,...
995,51.0,25.0,2004,11,2,non-working,2691.198920,0.998795
996,53.0,81.0,2010,12,2,working,2697.113429,0.996432
997,45.0,12.0,2009,9,23,working,2691.198920,0.998795
998,24.0,68.0,2005,10,18,working,4606.789040,0.233389


In [160]:
finalEval.sort_values(by=['demand'], ascending=True)

,drybulb,dewpnt,year,month,hour,non_working,demand,Fitness
401,54.0,65.0,2012,9,2,non-working,2688.182969,1.000000
524,48.0,17.0,2011,8,1,working,2688.182969,1.000000
220,62.0,-26.0,2015,5,4,working,2688.182969,1.000000
219,57.0,17.0,2011,8,1,working,2688.182969,1.000000
526,61.0,-13.0,2010,11,23,working,2688.182969,1.000000
...,...,...,...,...,...,...,...,...
819,89.0,-15.0,2004,2,23,non-working,4988.965844,0.080684
406,96.0,53.0,2008,6,23,working,4994.945063,0.078295
260,100.0,-16.0,2005,10,5,working,4994.945063,0.078295
732,89.0,45.0,2007,11,1,working,4994.945063,0.078295


In [168]:
top = finalEval[finalEval['Fitness'] == 1]
top

,drybulb,dewpnt,year,month,hour,non_working,demand,Fitness
2,56.0,15.0,2013,5,5,non-working,2688.182969,1.0
4,56.0,29.0,2010,3,2,working,2688.182969,1.0
13,61.0,67.0,2011,11,0,working,2688.182969,1.0
16,56.0,51.0,2013,5,1,working,2688.182969,1.0
24,60.0,-3.0,2011,4,4,working,2688.182969,1.0
...,...,...,...,...,...,...,...,...
987,47.0,72.0,2016,5,5,non-working,2688.182969,1.0
988,45.0,-21.0,2014,11,0,working,2688.182969,1.0
990,55.0,-27.0,2010,11,23,non-working,2688.182969,1.0
991,48.0,15.0,2013,6,4,working,2688.182969,1.0


In [170]:
top.describe()

,drybulb,dewpnt,year,month,hour,demand,Fitness
count,235.000000,235.000000,235.000000,235.000000,235.000000,2.350000e+02,235.0
mean,52.225532,32.655319,2012.714894,6.851064,8.893617,2.688183e+03,1.0
std,5.633783,32.203220,2.025371,2.834304,9.368601,3.190026e-12,0.0
min,41.000000,-27.000000,2010.000000,3.000000,0.000000,2.688183e+03,1.0
25%,47.500000,6.000000,2011.000000,4.000000,2.000000,2.688183e+03,1.0
50%,52.000000,30.000000,2012.000000,7.000000,4.000000,2.688183e+03,1.0
75%,56.000000,65.000000,2014.000000,9.500000,22.000000,2.688183e+03,1.0
max,62.000000,81.000000,2016.000000,11.000000,23.000000,2.688183e+03,1.0


In [166]:
initialDemand = predict_demand(pop_small)
initialEval = fitness(initialDemand)
initialEval
initialEval.describe()

,drybulb,dewpnt,year,demand,Fitness
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,51.545000,38.869000,2009.976000,3691.864709,0.703222
std,19.523861,20.203168,3.630786,700.565243,0.207150
min,-7.000000,-26.000000,2004.000000,2688.182969,0.000000
25%,36.000000,23.000000,2007.000000,3183.976571,0.627867
50%,52.000000,41.000000,2010.000000,3789.609699,0.674320
75%,67.000000,56.000000,2013.000000,3946.708982,0.853399
max,97.000000,74.000000,2016.000000,6070.109475,1.000000


In [167]:
pop_small

,drybulb,dewpnt,year,month,hour,non_working
0,52.0,48.0,2016,10,20,working
1,88.0,67.0,2012,8,13,working
2,30.0,29.0,2009,10,5,working
3,46.0,14.0,2012,4,9,non-working
4,31.0,7.0,2012,11,22,working
...,...,...,...,...,...,...
995,40.0,38.0,2007,11,4,non-working
996,83.0,67.0,2005,6,10,non-working
997,52.0,38.0,2014,10,16,working
998,77.0,72.0,2009,8,19,non-working
